This notebook is part of [**Byron v0.8**](https://pypi.org/project/byron/)  
Copyright 2023-24 Giovanni Squillero and Alberto Tonda  
SPDX-License-Identifier: [Apache-2.0](https://www.apache.org/licenses/LICENSE-2.0) 

In [1]:
import byron

/var/folders/31/dkl97hks2c14b663vl55pt440000gn/T/ipykernel_3752/1429530752.py:1: ByronPerformanceWarning: 
  Paranoia checks are enabled in this notebook: performances can be significantly impaired
  [see https://cad-polito-it.github.io/byron/paranoia for details]
  import byron


# Assembly Tutorial

Oversimplified assembler vaguely based on x86 assembly 

### Operands: Registers and Immediate 

In [2]:
reg8 = byron.f.choice_parameter(['ah', 'bh', 'ch', 'dh', 'al', 'bl', 'cl', 'dl'])
reg16 = byron.f.choice_parameter(['ax', 'bx', 'cx', 'dx'])
int8 = byron.f.integer_parameter(0, 2 ** 8)
int16 = byron.f.integer_parameter(0, 2 ** 16)

### Operands: Registers and Immediate

See https://docs.python.org/3/library/string.html#formatspec for the format specification

In [3]:
opcodes2 = byron.f.choice_parameter(['mov', 'add', 'sub', 'or', 'and'])
opcodes1 = byron.f.choice_parameter(['not', 'neg', 'inc', 'dec'])
opcodes0 = byron.f.choice_parameter(['nop', 'hlt'])

inst8rr = byron.f.macro("{op} {r1}, {r2}", op=opcodes2, r1=reg8, r2=reg8)
inst8ri = byron.f.macro("{op} {r}, {i:#x}", op=opcodes2, r=reg8, i=int8)
inst8r = byron.f.macro("{op} {r}", op=opcodes1, r=reg8)

inst16rr = byron.f.macro("{op} {r1}, {r2}", op=opcodes2, r1=reg16, r2=reg16)
inst16ri = byron.f.macro("{op} {r}, {i:#x}", op=opcodes2, r=reg16, i=int16)
inst16r = byron.f.macro("{op} {r}", op=opcodes1, r=reg16)

### A simple fragment of code

In [4]:
prog = byron.f.bunch([inst16rr, inst16ri, inst16r, inst8rr, inst8ri, inst8r], 5)
byron.f.as_text(prog)


; 🖋 n1 ➜ Frame❬MacroArray#1❭
and ax, ax  ; 🖋 n1.n2 ➜ Macro❬User#4❭
not ah  ; 🖋 n1.n3 ➜ Macro❬User#3❭
dec ax  ; 🖋 n1.n4 ➜ Macro❬User#6❭
and ax, 0x6ff1  ; 🖋 n1.n5 ➜ Macro❬User#5❭
sub dx, 0x65ce  ; 🖋 n1.n6 ➜ Macro❬User#5❭



### Let's add subroutines

In [5]:
sub_entry = byron.f.macro("\nproc {_node} near", _label='')
sub_exit = byron.f.macro("ret")
sub_body = byron.f.bunch([inst16rr, inst16ri, inst16r, inst8rr, inst8ri, inst8r], 3)
sub = byron.f.sequence([sub_entry, sub_body, sub_exit])

call = byron.f.macro("call {proc}", proc=byron.f.global_reference(sub, first_macro=True, creative_zeal=1))

Let's create a full program with header, initialization, and main body. Let "calls" be more frequent than the other instructions (62.5% of macros in `main_body` will be `call`). 

In [6]:
main_prologue = """section .text

global _start
_start:
"""

main_body = byron.f.bunch([inst16rr, inst16ri, inst16r, inst8rr, inst8ri, inst8r, call],
                          size=50,
                          weights=[1, 1, 1, 1, 1, 1, 10])
init = byron.f.macro("mov ax, {v:#x}\nmov bx, {v:#x}\nmov cx, {v:#x}\nmov dx, {v:#x}", v=int16)

full_prog = byron.f.sequence([main_prologue, init, main_body])

In [7]:
byron.f.as_text(full_prog)

; 🖋 n7 ➜ Frame❬FrameSequence#2❭
section .text

global _start
_start:
  ; 🖋 n7.n8 ➜ Macro❬Text#3❭
mov ax, 0x8e52
mov bx, 0x8e52
mov cx, 0x8e52
mov dx, 0x8e52  ; 🖋 n7.n9 ➜ Macro❬User#8❭
; 🖋 n7.n10 ➜ Frame❬MacroArray#3❭
or cx, bx  ; 🖋 n7.n10.n11 ➜ Macro❬User#4❭
call n62  ; 🖋 n7.n10.n12 ➜ Macro❬User#7❭
call n62  ; 🖋 n7.n10.n13 ➜ Macro❬User#7❭
call n69  ; 🖋 n7.n10.n14 ➜ Macro❬User#7❭
mov bh, 0x49  ; 🖋 n7.n10.n15 ➜ Macro❬User#2❭
and ch, dh  ; 🖋 n7.n10.n16 ➜ Macro❬User#1❭
not cx  ; 🖋 n7.n10.n17 ➜ Macro❬User#6❭
call n62  ; 🖋 n7.n10.n18 ➜ Macro❬User#7❭
sub dx, 0x46d3  ; 🖋 n7.n10.n19 ➜ Macro❬User#5❭
add ax, ax  ; 🖋 n7.n10.n20 ➜ Macro❬User#4❭
inc bx  ; 🖋 n7.n10.n21 ➜ Macro❬User#6❭
call n76  ; 🖋 n7.n10.n22 ➜ Macro❬User#7❭
call n76  ; 🖋 n7.n10.n23 ➜ Macro❬User#7❭
or bx, dx  ; 🖋 n7.n10.n24 ➜ Macro❬User#4❭
call n62  ; 🖋 n7.n10.n25 ➜ Macro❬User#7❭
call n76  ; 🖋 n7.n10.n26 ➜ Macro❬User#7❭
call n69  ; 🖋 n7.n10.n27 ➜ Macro❬User#7❭
call n62  ; 🖋 n7.n10.n28 ➜ Macro❬User#7❭
call n76  ; 🖋 n7.n10.n29 ➜ Macro❬